In [238]:
from __future__ import division
import math
import os
from collections import defaultdict
import re
import sys
import random
import itertools
from collections import Counter
from sklearn.preprocessing import normalize
import numpy as np
import itertools
import math
import scipy.misc


TRAIN_DIR = os.path.join(os.getcwd(),"Assignment2\\data\\train")
TEST_DIR = os.path.join(os.getcwd(),"Assignment2\\data\\test")
MODEL_DIR= os.path.join(os.getcwd(),"Assignment2\\model")

In [558]:
filenames=os.listdir(MODEL_DIR)

feature_gradient=np.loadtxt(os.path.join(MODEL_DIR,filenames[0]))
feature_params=np.loadtxt(os.path.join(MODEL_DIR,filenames[1]))
transition_gradient=np.loadtxt(os.path.join(MODEL_DIR,filenames[2]))
transition_params=np.loadtxt(os.path.join(MODEL_DIR,filenames[3]))
 
letterdict= {'e': 0, 't': 1, 'a': 2, 'i': 3, 'n': 4, 'o': 5, 's': 6, 'h': 7, 'r': 8, 'd': 9}    
trainfiles=os.listdir(TRAIN_DIR)
with open(os.path.join(TRAIN_DIR,'train_words.txt'),'r') as traindoc:
    train_words=traindoc.read().split()


testfiles=os.listdir(TEST_DIR)
with open(os.path.join(TEST_DIR,'test_words.txt'),'r') as testdoc:
    test_words=testdoc.read().split()

In [70]:
alphabet = list('etainoshrd')

In [67]:
# WF_cf
feature_params.shape


(10L, 321L)

In [68]:
# WT_cc'
transition_params.shape

(10L, 10L)

In [144]:
transition_params

array([[-0.371026,  0.120769, -0.194188, -0.385523,  0.107982, -0.197563,
         0.414634,  0.272522,  0.288212,  0.278577],
       [ 0.120769, -0.653739,  0.28069 ,  0.054133, -0.28821 ,  0.208675,
        -0.377716,  0.365483, -0.105408, -0.252407],
       [-0.194188,  0.28069 , -0.23071 ,  0.219636,  0.116347, -0.167909,
        -0.248713,  0.047337,  0.200981,  0.066791],
       [-0.385523,  0.054133,  0.219636, -0.328905,  0.306496, -0.026209,
         0.286039, -0.07772 ,  0.22133 ,  0.01934 ],
       [ 0.107982, -0.28821 ,  0.116347,  0.306496, -0.032171,  0.125915,
         0.009618, -0.117396, -0.089888,  0.098682],
       [-0.197563,  0.208675, -0.167909, -0.026209,  0.125915,  0.198127,
        -0.134644,  0.008058,  0.220512,  0.028377],
       [ 0.414634, -0.377716, -0.248713,  0.286039,  0.009618, -0.134644,
        -0.320125, -0.009717, -0.322828, -0.005847],
       [ 0.272522,  0.365483,  0.047337, -0.07772 , -0.117396,  0.008058,
        -0.009717, -0.136171, -0.0900

In [559]:
feature_gradient.shape

(10L, 321L)

In [71]:
transition_gradient.shape

(10L, 10L)

In [43]:
X_train=[]
X_test=[]

In [44]:
for i in range(len(train_words)):
    X_train.append(np.loadtxt(os.path.join(TRAIN_DIR,"train_img"+str(i+1)+".txt")))


In [45]:
X_train[2].shape

(5L, 321L)

In [46]:
for i in range(len(test_words)):
    X_test.append(np.loadtxt(os.path.join(TEST_DIR,"test_img"+str(i+1)+".txt")))

In [56]:
X_test[0].shape

(4L, 321L)

In [4]:
letterdict['e']

0

In [61]:
Y_train=[]
Y_test=[]

for i in range(len(train_words)):
    Y_train.append(list(train_words[i]))
    
for i in range(len(test_words)):
    Y_test.append(list(test_words[i]))

In [73]:
X_test[0]

array([[ 1.,  1.,  1., ...,  1.,  0.,  1.],
       [ 1.,  1.,  1., ...,  0.,  1.,  1.],
       [ 1.,  0.,  0., ...,  0.,  0.,  0.],
       [ 1.,  1.,  0., ...,  1.,  1.,  0.]])

In [110]:
Y_test[0]

['t', 'h', 'a', 't']

In [164]:
def node_potentials(word, label_sequence ,feature_params):
# feature_params,  k x n numpy array of feature parameter.
# word ,a w x 1 numpy array of feature vectors,
    calc= np.dot(word, np.transpose(feature_params))
    potentials=np.zeros((len(label_sequence),1))
    for i in range(len(label_sequence)):
        potentials[i]= calc[i][alphabet.index(label_sequence[i])]
    return np.exp(potentials)

In [165]:
node_that=node_potentials(X_test[0],Y_test[0],feature_params)
node_that

array([[  1.04885517e+08],
       [  2.20913251e+01],
       [  2.92388248e+07],
       [  4.50596498e+10]])

In [166]:
np.log(node_that)

array([[ 18.46838 ],
       [  3.095185],
       [ 17.191008],
       [ 24.531253]])

In [124]:
from sklearn.preprocessing import normalize
node_norm= normalize(node_that, norm='l1',axis=1)
sum(node_norm[3])


1.0

In [502]:
def log_partition(X_test, Y_test,feature_params,transition_params,n):
# Exhaustive sum of the all label sequences for a word
    # First three words
    exhaustive_sum=[]

    for k in range(n):
        permutations= list(itertools.product(alphabet,repeat =len(Y_test[k])))
        feature_sum=0
        transition_sum=0
        e_sum=0
        for j in range(len(permutations)): 
            feat=node_potentials(X_test[k],permutations[j],feature_params)
            feature_sum= reduce(lambda x,y: x*y,feat) 
            tran=(np.exp([transition_params[alphabet.index(permutations[j][i])][alphabet.index(permutations[j][i+1])] for i in range(len(permutations[j])-1)]))
            transition_sum=reduce(lambda x,y: x*y,tran)
            
            e_sum+=(feature_sum*transition_sum)
        exhaustive_sum.append(e_sum) 
    return (exhaustive_sum)


In [508]:
es=log_partition(X_test, Y_test,feature_params,transition_params,3)
print "Exhaustive sum for three words is : \n",es
print "In log scale \n",np.log(es)


Exhaustive sum for three words is : 
[array([  2.28623899e+29]), array([  8.29939019e+38]), array([  9.15068082e+44])]
In log scale 
[[  67.6018758 ]
 [  89.61441558]
 [ 103.52757238]]


In [275]:
def energy (x, y,feature_params,transition_params):
# Compute energy defined in question 1.2
# inputs: input x, label sequence y, params
# output: energy (a number)
    node_pot=np.log(node_potentials(x,y,feature_params))
    feature_sum= np.sum(node_pot)
    transition_sum=sum([transition_params[alphabet.index(y[i])][alphabet.index(y[i+1])] for i in range(len(y)-1)])
    return (feature_sum+transition_sum)

In [276]:
E1=energy(X_test[0],Y_test[0],feature_params,transition_params)
E2=energy(X_test[1],Y_test[1],feature_params,transition_params)
E3=energy(X_test[2],Y_test[2],feature_params,transition_params)

print " Negative energy of the true label sequence, ",Y_test[0],"is :",E1
print " Negative energy of the true label sequence, ",Y_test[1],"is :",E2
print " Negative energy of the true label sequence, ",Y_test[2],"is :",E3

 Negative energy of the true label sequence,  ['t', 'h', 'a', 't'] is : 63.979336
 Negative energy of the true label sequence,  ['h', 'i', 'r', 'e'] is : 89.61093
 Negative energy of the true label sequence,  ['r', 'i', 's', 'e', 's'] is : 96.940634


In [514]:
def feature_potentials(word,feature_params):
# feature_params, C x n numpy array of feature parameter.
# word ,a w x n numpy array of feature vectors,
# word= X_test[i]
    return np.dot(word, np.transpose(feature_params))
    

def transition_potential (feat_pot1,feat_pot2, transition_params):
# Absorb node potential into a pairwise potential,
# for positions (t, t+1).
# output: (log) pairwise potential function.  (a table, e.g. array)
# Returns a C x C numpy array, where k is the size of the alphabet.
    tran_pot = transition_params+ feat_pot1[:, np.newaxis]
    if feat_pot2 is not None:
        tran_pot += feat_pot2
    return tran_pot

def chain_potentials(word,feature_params,transition_params):
#Computes the clique potentials of the entire chain.
#Returns a (w-1) x C x C numpy array,
    phi = feature_potentials(word, feature_params)
    
    # Include the potentials of the last two nodes in the same clique
    transitions = [(node, None) for node in phi[:-2]] + [(phi[-2], phi[-1])]
    #transitions = [(node, None) for node in phi[:]]
    psi = [transition_potential(node1, node2, transition_params) for node1, node2 in transitions]
    return np.array(psi)


def message_passing (psi):
# Message passing algorithm
# input: (log-) potential
# outputs: forward/backward messages
    # Backward messages
    back = []
    prev_msgs = np.zeros(psi.shape[1])
    for pairs in psi[:0:-1]:
        message = scipy.misc.logsumexp(pairs + prev_msgs, axis=1)
        back.append(message)
        prev_msgs += message

    # Forward messages
    fwd = []
    prev_msgs = np.zeros(psi.shape[1])
    for pairs in psi[:-1]:
        message = scipy.misc.logsumexp(pairs + prev_msgs[:, np.newaxis], axis=0)
        fwd.append(message)
        prev_msgs += message

    return (np.array(back), np.array(fwd))



In [515]:
psi_that=chain_potentials(X_test[0],feature_params,transition_params)
back,forward=message_passing(psi_that)
print "Forward message array : \n", (forward)
print "Backward message array : \n", (back)
print "\n \n"
print " Log-space message m1→2(Y2) : ", (forward[0][alphabet.index(Y_test[0][1])])
print " Log-space message m2→1(Y1) : ", (back[0][alphabet.index(Y_test[0][0])])
print " Log-space message m2→3(Y3) : ", (forward[1][alphabet.index(Y_test[0][2])])
print " Log-space message m3→2(Y2) : ", (back[1][alphabet.index(Y_test[0][1])])

Forward message array : 
[[ 18.58934459  17.81529477  18.74937334  18.52273392  18.18075315
   18.67731044  18.0912883   18.83407002  18.3634186   18.21639565]
 [ 25.65107866  25.23685908  25.59838345  25.57794329  25.27163715
   25.60124492  25.07146009  25.38802664  25.4145125   25.20264414]]
Backward message array : 
[[ 14.4438841   24.77485617  42.00295108  12.5677121   29.82240312
   24.14588509   2.72717112  34.04564006  33.90826111  26.22602211]
 [ 37.73533939  48.02906741  42.94954937  40.43003538  40.90760095
   40.05103627  33.45509765  45.14603484  49.0110484   42.41188976]]

 

 Log-space message m1→2(Y2) :  18.834070015
 Log-space message m2→1(Y1) :  24.7748561679
 Log-space message m2→3(Y3) :  25.5983834534
 Log-space message m3→2(Y2) :  45.1460348364


In [365]:
def beliefs(word,feature_params,transition_params):
# Returns a numpy array of size (w-1) x k x k,
    psi = chain_potentials(word,feature_params,transition_params)
    delta_bwd, delta_fwd = message_passing(psi)

    k = delta_fwd.shape[1]
    delta_fwd = np.concatenate(([np.zeros(k)], delta_fwd))
    delta_bwd = np.concatenate((delta_bwd[::-1], [np.zeros(k)]))
    belief = psi + delta_fwd[:, :, np.newaxis] + delta_bwd[:, np.newaxis]

    return np.array(belief)

In [428]:
def pairwise_prob(belief):
# pairwise marginal probabilities.
    return np.exp(belief - scipy.misc.logsumexp(belief, axis=(1,2))[:, np.newaxis, np.newaxis])

def single_prob(pairwise_p):
# singleton marginal probabilities.

    a = np.sum(pairwise_p, axis=2)
    b = np.sum(pairwise_p[-1], axis=0) # Last character in the word

    return np.concatenate((a, b[np.newaxis, :]))

def joint_prob(single_p, label_sequence, alphabet):
# Computes the joint probability of the label given singleton marginal probabilities.
    prob = [np.log(marginal[alphabet.index(c)]) for (c, marginal) in zip(label_sequence, single_p)]

    return np.exp(np.sum(prob))



In [536]:
belief_that= beliefs(X_test[0],feature_params,transition_params)
pw_prob= pairwise_prob(belief_that)
s_prob=single_prob(pw_prob)
pw_prob.shape

(3L, 10L, 10L)

In [533]:
pw_prob[0][[1,2,7],:][:,[1,2,7]]

array([[  1.72360443e-01,   2.73053851e-03,   2.67297536e-02],
       [  7.46583831e-12,   2.78595262e-14,   3.30864004e-13],
       [  1.59042715e-11,   7.20009247e-14,   5.38968123e-13]])

In [534]:
pw_prob[1][[1,2,7],:][:,[1,2,7]]

array([[  2.23141736e-09,   1.72371206e-01,   6.56861166e-05],
       [  1.49969782e-10,   2.72880257e-03,   1.26156644e-06],
       [  1.21044035e-09,   2.67202731e-02,   7.78624591e-06]])

In [535]:
pw_prob[2][[1,2,7],:][:,[1,2,7]]

array([[  2.29451163e-08,   2.07955205e-24,   1.05809723e-21],
       [  9.99458796e-01,   2.13367796e-17,   1.31708499e-14],
       [  2.83525311e-04,   7.34319323e-21,   2.85705826e-18]])

In [445]:
def likelihood_true(Y_train,X_train,alphabet,feature_params,transition_params,n):
# Objective function to minimize.
# Returns the negative average log likelihood of theta given the data.    
    p = []
    for i in range(len(Y_train[:n])):
        belief = beliefs(X_train[i],feature_params,transition_params)
        pairwise_p = pairwise_prob(belief)
        single_p = single_prob(pairwise_p)
        p.append(joint_prob(single_p,Y_train[i] , alphabet))
    print p
    return np.sum(p)/len(Y_train[:n])


def likelihood_predict(Y_train,X_train,alphabet,feature_params,transition_params,n):
# Objective function to minimize.
# Returns the negative average log likelihood of theta given the data.    
    p = []
    for i in range(len(Y_train[:n])):
        belief = beliefs(X_train[i],feature_params,transition_params)
        pairwise_p = pairwise_prob(belief)
        single_p = single_prob(pairwise_p)
        p.append(joint_prob(single_p, predict_word(single_p, alphabet), alphabet))
    print p
    return np.sum(p)/len(Y_train[:n])


In [511]:
likelihood_true(Y_train,X_train,alphabet,feature_params,transition_params,50)

[0.97558341038452212, 0.99050814094478423, 0.00014428552955861841, 0.91662951088527911, 0.0053845825352322489, 0.007200290545782956, 0.99986641183040836, 0.002346236423977373, 2.6476486787640712e-07, 0.82023082117478052, 0.99988316906039953, 0.98511477871051001, 0.99433901609018771, 8.6020530317901239e-06, 0.076872350577993326, 2.2497166501472388e-06, 0.010336764665831675, 0.99790197753269594, 0.0052825534264687264, 1.8029298336690324e-09, 0.45815171980553171, 1.3252514148760273e-09, 0.85112901832845433, 0.42832926543684463, 2.6571939438687889e-06, 0.0017449701514608579, 0.044700394613537899, 0.99991281455599279, 0.02685200858821158, 0.91901107835611895, 0.99945352815662114, 0.00034387713967458941, 0.00015279635546277601, 1.9920725149157764e-05, 0.9959997341665715, 0.014044941027658409, 0.032465560296249751, 0.0032558355816338313, 0.99997897743278374, 0.99714389519924396, 0.99999544122737394, 0.99988663980724113, 0.99631869122263228, 0.00024119451419604357, 8.156979416313076e-06, 8.421

0.43728256154790235

In [446]:
likelihood_predict(Y_test,X_test,alphabet,feature_params,transition_params,3)

[0.79578548353809586, 0.99652049240843565, 0.93700663528992467]


0.9097708704121521

In [377]:
def predict_word(single_p, alphabet):
# Returns a list of predicted characters of a word.
# Parameters:
#    - single_p, a w x k numpy array of singleton marginal probabilities,
#      where w is the word length and k is the size of the alphabet; and
#    - alphabet, a list of all possible character labels.
    indices = np.argmax(single_p, axis=1)
    
    return [alphabet[i] for i in indices]

def predict(Y_train,X_train,alphabet,feature_params,transition_params):
# Returns a list of predictions, where each prediction is a list of predicted character labels of a word.
    predictions = []
    for i in range(len(Y_train)):
        belief = beliefs(X_train[i],feature_params,transition_params)
        pairwise_p = pairwise_prob(belief)
        single_p = single_prob(pairwise_p)
        predictions.append(predict_word(single_p, alphabet))

    return predictions

In [510]:
belief_that= beliefs(X_test[0],feature_params,transition_params)
pw_prob1= pairwise_prob(belief_that)
s_prob1=single_prob(pw_prob1)

belief_hire= beliefs(X_test[1],feature_params,transition_params)
pw_prob2= pairwise_prob(belief_hire)
s_prob2=single_prob(pw_prob2)

belief_rises= beliefs(X_test[2],feature_params,transition_params)
pw_prob3= pairwise_prob(belief_rises)
s_prob3=single_prob(pw_prob3)
print "Predicted sequence for word 1 is : ",predict_word(s_prob1, alphabet),"with probability : ",joint_prob(s_prob1, predict_word(s_prob1, alphabet), alphabet)
print "Predicted sequence for word 2 is : ",predict_word(s_prob2, alphabet),"with probability : ",joint_prob(s_prob2, predict_word(s_prob2, alphabet), alphabet)
print "Predicted sequence for word 3 is : ",predict_word(s_prob3, alphabet),"with probability : ",joint_prob(s_prob3, predict_word(s_prob3, alphabet), alphabet)



Predicted sequence for word 1 is :  ['t', 'r', 'a', 't'] with probability :  0.795785483538
Predicted sequence for word 2 is :  ['h', 'i', 'r', 'e'] with probability :  0.996520492408
Predicted sequence for word 3 is :  ['r', 'i', 's', 'e', 'r'] with probability :  0.93700663529


In [351]:
predict(Y_train,X_train,alphabet,feature_params,transition_params)

[['t', 'h', 'e', 'o', 'd', 'o', 'r', 'e', 's'],
 ['i', 'i', 't', 's', 'e'],
 ['n', 'i', 'a', 't', 'h', 't'],
 ['h', 'a', 'n', 's', 'e'],
 ['s', 'i', 'a', 'e', 's'],
 ['i', 'r', 't', 's', 'h', 'e'],
 ['e', 'r', 'i', 't', 'r', 'e', 'a', 't'],
 ['s', 'e', 'a', 'e', 'o', 'n', 'i'],
 ['s', 'n', 'e', 'e', 't', 'e'],
 ['s', 'a', 'n', 't', 'o', 's', 'e'],
 ['t', 'h', 'e', 's'],
 ['h', 'a', 'r', 't', 'h'],
 ['n', 'o', 't', 'i', 'o', 'n', 'i'],
 ['a', 'r', 't', 'r', 'e', 's'],
 ['h', 'a', 'a', 't'],
 ['i', 'n', 'r', 't', 'e', 'a', 'd', 'e'],
 ['a', 'o', 's', 'a', 's', 's', 'i', 'd', 'a', 't', 'i', 'o', 'n', 'e'],
 ['t', 'e', 'r', 'r', 'a', 'i', 'n', 'i'],
 ['s', 't', 'd', 'n', 'e', 's', 'e'],
 ['e', 'a', 's', 't', 'e', 'r', 'e'],
 ['h', 'i', 'r', 'd', 'e'],
 ['i', 't', 'd', 'o', 't', 'e', 's', 'i', 'h', 'r', 'e'],
 ['r', 'e', 's', 'i', 'd', 'e', 's'],
 ['t', 'n', 'o', 's', 'e', 's'],
 ['t', 'n', 't', 'n', 'i', 'o', 'r'],
 ['s', 'r', 'i', 'n'],
 ['i', 'h', 'a', 't', 'h'],
 ['t', 'i', 'e', 's'],
 

In [415]:
predict(Y_test,X_test,alphabet,feature_params,transition_params)[:5]

[['t', 'r', 'a', 't'],
 ['h', 'i', 'r', 'e'],
 ['r', 'i', 's', 'e', 'r'],
 ['e', 'd', 'i', 's', 'o', 'n'],
 ['s', 'h', 'o', 'r', 'e']]

In [384]:
Y_test[:5]

[['t', 'h', 'a', 't'],
 ['h', 'i', 'r', 'e'],
 ['r', 'i', 's', 'e', 's'],
 ['e', 'd', 'i', 's', 'o', 'n'],
 ['s', 'h', 'o', 'r', 'e']]

In [417]:
def prediction_accuracy(Y_test,X_test,alphabet,feature_params,transition_params,n):
# Returns accuracy of the sequence predcition task
# Limiting to n words
    predictions= predict(Y_test,X_test,alphabet,feature_params,transition_params)
    total=0
    correct=0
    for i in range(n):
        total+=len(Y_test[i])
        count=0
        for j in range(len(Y_test[i])):
            if predictions[i][j]==Y_test[i][j]:
                count+=1
        correct+= count
    return 100*correct/total

In [418]:
prediction_accuracy(Y_test,X_test,alphabet,feature_params,transition_params,5)

91.66666666666667

In [567]:
def feature_grad(Y_train,X_train,alphabet,feature_params):
# Returns a flattened k x n numpy array,where k is the size of the alphabet and n is the length of the feature vector.


    # Initialize a state gradient table of size k x n with zeros
    gradient = np.zeros((len(alphabet), len(X_train[0][1])))

    for i in range(len(Y_train[:50])):
        belief = beliefs(X_train[i],feature_params,transition_params)
        pairwise_p = pairwise_prob(belief)
        single_p = single_prob(pairwise_p)
        for v, c, p in zip(X_train[i], Y_train[i], single_p):
            for i in range(gradient.shape[0]): # possible labels
                for j in range(gradient.shape[1]): # features
                    indicator = 0
                    if c == alphabet[i]:
                        indicator = 1
                    gradient[i][j] += (indicator - p[i]) * v[j]
    
    gradient /= len(Y_train[:50])

    return np.ndarray.flatten((gradient))

def transition_grad(Y_train,X_train,alphabet,feature_params,transition_params):
# Returns a flattened k x k numpy array, where k is the size of the alphabet.

    # Initialize a transition gradient table of size k x k with zeros
    gradient = np.zeros((len(alphabet), len(alphabet)))

    for i in range(len(Y_train[:50])):
        belief = beliefs(X_train[i],feature_params,transition_params)
        pairwise_p = pairwise_prob(belief)
        label_pairs = zip([None] + Y_train[i], Y_train[i] + [None])[1:-1]

        for (label1, label2), p in zip(label_pairs, pairwise_p):
            for i in range(gradient.shape[0]):
                for j in range(gradient.shape[1]):
                    indicator = 0
                    if label1 == alphabet[i] and label2 == alphabet[j]:
                        indicator = 1
                    gradient[i][j] += indicator - p[i][j]

    gradient /= len(Y_train[:50])

    return np.ndarray.flatten((gradient))

def likelihood_prime(Y_train,X_train,alphabet,feature_params,transition_params):
# Returns a flattened numpy array of the [feature_gradient, transition_gradient] list.

    return np.concatenate((feature_grad(Y_train,X_train,alphabet,feature_params),
                           transition_grad(Y_train,X_train,alphabet,feature_params,transition_params)))



In [570]:
calc_featgrad=feature_grad(Y_train,X_train,alphabet,feature_params)

In [571]:
calc_featgrad[:10]


array([-0.00790173, -0.00741463, -0.00733574, -0.00708563, -0.00152063,
       -0.0214657 , -0.00189108, -0.00608977, -0.00565361, -0.00741145])

In [569]:
feature_gradient[0][:10]

array([-0.00790398, -0.00741687, -0.00733822, -0.00708787, -0.00152144,
       -0.02146795, -0.00189334, -0.00609199, -0.00565587, -0.00741369])

In [227]:
import scipy.optimize as optimize
def func(x, sign=-1.0):
    return sign*(-(1-x[0])**2 - 100*(x[1]-x[0]**2)**2)

def func_deriv(x, sign=1.0):
    """ Derivative of objective function """
    dfdx0 = sign*(2*(1-x[0]) + 400*x[0]*(x[1]-x[0]**2))
    dfdx1 = sign*(-200*(x[1]-x[0]**2))
    return np.array([ dfdx0, dfdx1 ])

res = optimize.minimize(func, [-1.0,1.0], args=(-1.0,), jac=func_deriv,method='SLSQP', options={'disp': True})
res.x


Optimization terminated successfully.    (Exit mode 0)
            Current function value: 1.49996772796e-09
            Iterations: 39
            Function evaluations: 54
            Gradient evaluations: 39


array([ 0.99996418,  0.99992689])